# Engineer's Thesis

This notebook shows only VGG19 training for preview

### import dataset

In [ ]:
# tensorflow loggin disabled
import datetime, logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import tensorflow 
tf.get_logger().setLevel('ERROR')

In [ ]:
import os

path = ''
data_path = path + 'data/'
model_path = path + 'models/'

In [ ]:
import tensorflow_datasets as tfds

df = tfds.load(name="stanford_dogs", split='train+test')
print(df)

### Conf and helper functions

In [ ]:
batch_size = 64
img_size = 224
n_classes = 120
seed = 61

In [ ]:
import keras.backend as K
import numpy as np

# Function counting classes distribution
def count_class_distribution(ds):
  count = np.zeros(shape=n_classes)
  c = 0
  for example in ds.take(-1):
    label = example["label"]
    count[tf.keras.backend.get_value(label)] +=1
    c+=1
  return count, c

In [ ]:
class_count, n_examples = count_class_distribution(df)

# Count distribution matrix
test_count = class_count
distribution = test_count/n_examples

In [ ]:
splits = []

# Create subsets, one for every class
for i in range(0, (n_classes)):
  dataset = df.filter(lambda fd: fd['label'] == i)
  splits.append(dataset)

In [ ]:
test_ds_size = 1000
validation_split = 0.8

# Create training, validation and test set
def get_smaller_ds(ds, splits, size):
  train_ds = df.take(0)
  val_ds = df.take(0)
  test_ds = df.take(0)

  for d, s in zip(distribution, splits):
    n_test_ds = int(d * test_ds_size)
    n_train_ds = int(d * (size * validation_split))
    n_val_ds = int(d * (size * (1 - validation_split)))
    
    # test
    test_result = s.take(n_test_ds)
    test_ds = test_ds.concatenate(test_result)

    #train
    train_result = s.skip(n_test_ds)
    train_result = train_result.take(n_train_ds)
    train_ds = train_ds.concatenate(train_result)

    #val
    val_skip = n_test_ds + n_train_ds
    val_result = s.skip(val_skip)
    val_result = val_result.take(n_val_ds)
    val_ds = val_ds.concatenate(val_result)

  return train_ds, val_ds, test_ds

In [ ]:
def preprocess(ds_row):
    # Image conversion int->float + resizing
    #image = tf.image.convert_image_dtype(ds_row['image'], dtype=tf.float32)
    image = tf.image.resize(ds_row['image'], (img_size, img_size), method='nearest')
  
    # Onehot encoding labels
    label = tf.one_hot(ds_row['label'], n_classes)

    return image, label

In [ ]:
def prepare(dataset, batch_size):
    ds = dataset.map(preprocess, num_parallel_calls=4)
    ds = ds.shuffle(buffer_size=(n_examples*2), seed=seed)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

## Create models

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.layers import Dropout, Lambda, BatchNormalization
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D

In [ ]:
# Augmentation layer
augmentation = models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="augmentation",
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# Function that plots metrics after training
def plot_metrics(name, history, epochs, lr):  
  history_accuracy = {
      'accuracy' : history.history['accuracy'],
      'val_accuracy' : history.history['val_accuracy']
  }
  history_loss = {
      'loss' : history.history['loss'],
      'val_loss' : history.history['val_loss']
  }

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.grid(True)
  plt.title(name + '     Accuracy metrics     Learning rate: ' + str(lr))
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  # -----
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.grid(True)
  plt.title(name + '     Loss metrics     Learning rate: ' + str(lr))
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [ ]:
# Create list of callbacks
def list_of_callbacks(name, learning_rate):
  filepath = model_path + name  + 'lr' + str(learning_rate)

    # Save model callback
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=filepath,
      save_weights_only=False,
      monitor='val_accuracy',
      verbose=0,
      save_freq='epoch',
      save_best_only=False)
  
  # Save history callback
  history_logger=tf.keras.callbacks.CSVLogger(filepath + '.csv', 
                                              separator=",", 
                                              append=True)
  
  # Early stopping if no progress or bad at start
  early = tf.keras.callbacks.EarlyStopping(monitor='accuracy', 
                                           min_delta=0.01, 
                                           patience=4, 
                                           verbose=0, 
                                           mode='auto', 
                                          #  baseline=0.7, 
                                           restore_best_weights=False)
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, 
                                                        histogram_freq=1)

  return [history_logger, tensorboard_callback, model_checkpoint_callback]

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

# Compile model -> train model -> plot metrics after training
def test_model(name, model, train_data, valid_data, learning_rate):
  print("Model name:  ", name, " learning rate:  ", str(learning_rate))

  model.compile(optimizer=Adam(learning_rate=learning_rate), 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

  history = model.fit(train_data,
                    validation_data=valid_data,
                    epochs=epochs,
                    callbacks=list_of_callbacks(name, learning_rate))
  
  plot_metrics(name, history, epochs, learning_rate)

  return model

In [ ]:
# Model evaluation
def eval(name, model, train_batches, val_batches, test_batches, lr_val):
  model_array = np.empty(shape=(len(lr_val), 2))
  # import initial weights (prevents starting training with trained weights)
  weights = model.get_weights()
  
  # train for different learning rates 
  count = 0
  for i in lr_val:
    model.set_weights(weights)
    m = test_model(name, model, train_batches, val_batches, i)
    r = m.evaluate(test_batches)
    #Debug
    print(r)
    model_array[count] = r
    count += 1
    nam = name + str(i) + '.csv'
    np.savetxt(nam, model_array, delimiter=',')

  return model_array

In [ ]:
# Helper function which creates model with preprocessing layer 
def create_model(base_model, preprocess):
  base_model.trainable = False

  preprocess_layer = Lambda(preprocess, 
                          name='preprocessing', 
                          input_shape=(img_size, img_size, 3))

  model = models.Sequential(
    [
     keras.Input(shape=(img_size, img_size, 3)),
     augmentation,
     preprocess_layer,

     base_model,
     GlobalAveragePooling2D(),
     Dense(n_classes, activation = 'softmax', name='output')
    ]
  )
  model.summary()

  return model

## Training

In [ ]:
import time

epochs = 25
lr = 0.0005
train_ds_size = 4000

train_data, val_data, test_data = get_smaller_ds(df, splits, train_ds_size)

train_batches = prepare(train_data, batch_size=batch_size)
val_batches = prepare(val_data, batch_size=batch_size)
test_batches = prepare(test_data, batch_size=batch_size)

## VG19 4000

Divide each training for separate cell for safety

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19

row_XC = pd.DataFrame(columns=['loss','accuracy', 'training_time', 'test_time'])
name = ("VGG19" + str(train_ds_size) + '.csv')
row_XC.to_csv(name)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

## VGG19 8000

In [ ]:
train_ds_size = 8000

train_data, val_data, test_data = get_smaller_ds(df, splits, train_ds_size)

train_batches = prepare(train_data, batch_size=batch_size)
val_batches = prepare(val_data, batch_size=batch_size)
test_batches = prepare(test_data, batch_size=batch_size)

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19

row_XC = pd.DataFrame(columns=['loss','accuracy', 'training_time', 'test_time'])
name = ("VGG19" + str(train_ds_size) + '.csv')
row_XC.to_csv(name)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

## VGG19 12000

In [ ]:
train_ds_size = 12000

train_data, val_data, test_data = get_smaller_ds(df, splits, train_ds_size)

train_batches = prepare(train_data, batch_size=batch_size)
val_batches = prepare(val_data, batch_size=batch_size)
test_batches = prepare(test_data, batch_size=batch_size)

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19

row_XC = pd.DataFrame(columns=['loss','accuracy', 'training_time', 'test_time'])
name = ("VGG19" + str(train_ds_size) + '.csv')
row_XC.to_csv(name)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)

## VGG19 20000

In [ ]:
train_ds_size = n_examples

train_data, val_data, test_data = get_smaller_ds(df, splits, train_ds_size)

train_batches = prepare(train_data, batch_size=batch_size)
val_batches = prepare(val_data, batch_size=batch_size)
test_batches = prepare(test_data, batch_size=batch_size)

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19

row_XC = pd.DataFrame(columns=['loss','accuracy', 'training_time', 'test_time'])
name = ("VGG19" + str(train_ds_size) + '.csv')
row_XC.to_csv(name)

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)
# 11 epochs gone

In [ ]:
base_model = VGG19(weights = 'imagenet', 
                        include_top = False, 
                        input_shape = (img_size, img_size, 3))

model = create_model(base_model, 
                        tensorflow.keras.applications.vgg19.preprocess_input)
  
start_t = time.time()
m = test_model(name, model, train_batches, val_batches, lr)
end_t = time.time()

start_e = time.time()
r = m.evaluate(test_batches)
end_e = time.time()

row_XC = row_XC.append(pd.DataFrame([[r[0], r[1], (end_t-start_t), (end_e-start_e)]], 
                    columns=['loss','accuracy', 'training_time', 'test_time']))

r = pd.read_csv(name)

r = pd.concat([r,row_XC])
r.to_csv(name, index=False)